## KAFKA SETUP

In [1]:
!pip install kafka-python
import kafka
import time

## MONGO SETUP

In [2]:
!pip install pymongo
import pymongo
myclient = pymongo.MongoClient('mongo', 27017, username='admin', password='DataMan2019!')
database = myclient["nba"]
collection = database["sunsbucks0"]

## TWITTER API SETUP

In [3]:
!pip install tweepy
import tweepy

# outputs the api
def twitter_connect():
    
    # Set consumer key, consumer secret, access token and access token secret
    # Read them from your Twitter dev page
    consumer_key = "C9yF34rMp0srBjvnitisAm08f"
    consumer_secret = "wWq2KOtWZYZfM48kL9dX31YFjhofZwgGGcf1vaekbGyhzqg7Z1"
    access_token = "1304145794149355521-Yj69Z6xVbbdgCLTWEIGpzuzBAeWeCj"
    access_token_secret = "u4zDDvDVDSpDbYY0SGE2lldb0o8dmnrEkePxvI3k15GHt"

    # Authenticate with the OAuth protocol
    # See: https://en.wikipedia.org/wiki/OAuth
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    # Get the API object passing the authentication object
    # See http://docs.tweepy.org/en/latest/api.html to discover the available methods
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

api = twitter_connect()

# check connection
print(api.home_timeline(count=1)[0].text)

#LERCIOSTORY Si rasano per solidarietà con amico malato di cancro, ma era diventato ultrà della Lazio… https://t.co/r2y8HTp42V


## STREAMING TWEETS TO KAFKA

In [4]:
from datetime import datetime, timedelta

def normalize_timestamp(time):
    mytime = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    mytime += timedelta(hours=2)   # the tweets are timestamped in GMT timezone, while I am in +2 timezone
    return (mytime.strftime("%Y-%m-%d %H:%M:%S"))

import json

# create kafka producer
producer = kafka.KafkaProducer(bootstrap_servers=['kafka:9092'], value_serializer=lambda v: json.dumps(v).encode("utf-8"))
# connecting stream to kafka
class KafkaStreamListener(tweepy.streaming.StreamListener):
        
    def __init__(self, time_limit=10):
        self.start_time=time.time()
        self.limit = time_limit
        super(KafkaStreamListener, self).__init__() 
        
    def on_status(self, status):
        try:
            #print(status.retweeted_status.user.name)
            producer.send('sunsbucks0', value={'id': status.id_str,
                                        'user_location':status.user.location,
                                        'language':status.lang,
                                        'timestamp':normalize_timestamp(str(status.created_at)),
                                        'text':status.text,
                                        'retweet_count':status.retweet_count,
                                        'user_followers':status.user.followers_count,
                                        'geo':status.geo,
                                        'coordinates':status.coordinates,
                                        'retweeted':status.retweeted,
                                        'is_a_retweet': str(x),
                                        'user_mentions':status.entities['user_mentions'],
                                        'entities_hashtags':status.entities['hashtags'],
                                        'in_reply_to_status_id_str':status.in_reply_to_status_id_str,
                                        'in_reply_to_user_id_str':status.in_reply_to_user_id_str,
                                        'retweeted_status_id':status.retweeted_status.id_str,
                                        'retweeted_status__user_id':status.retweeted_status.user.id_str,
                                        'retweeted_status_user_name':status.retweeted_status.user.name,
                                        'id_str':status.user.id_str,
                                        'screen_name':status.user.screen_name})
        except:
            producer.send('sunsbucks0', value={'id': status.id_str,
                                        'user_location':status.user.location,
                                        'language':status.lang,
                                        'timestamp':normalize_timestamp(str(status.created_at)),
                                        'text':status.text,
                                        'retweet_count':status.retweet_count,
                                        'user_followers':status.user.followers_count,
                                        'geo':status.geo,
                                        'coordinates':status.coordinates,
                                        'retweeted':status.retweeted,
                                        'is_a_retweet': str(x),
                                        'user_mentions':status.entities['user_mentions'],
                                        'entities_hashtags':status.entities['hashtags'],
                                        'in_reply_to_status_id_str':status.in_reply_to_status_id_str,
                                        'in_reply_to_user_id_str':status.in_reply_to_user_id_str,
                                        'id_str':status.user.id_str,
                                        'screen_name':status.user.screen_name})
      
# connecting kafka to mongo      
consumer = kafka.KafkaConsumer("sunsbucks0", bootstrap_servers=['kafka:9092'], auto_offset_reset='earliest', 
                               value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [5]:
# threading

import logging
import threading
import time

# just a handy formatting of the logs
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

def producer_function():
    while True:
        logging.info(f'producer thread: starting')
        stream = tweepy.Stream(api.auth, listener=KafkaStreamListener())
        stream.filter(track=['#NBAFinals', '#FearTheDeer', '#RallyTheValley', '#Suns', '#Bucks'])
        logging.info(f'producer thread: exception') # to show this line the sleep should be here not on main thread

def consumer_function():
    while True:
        logging.info(f'consumer thread: starting')
        for message in consumer:
            collection.insert_one(message.value)
        logging.info(f'consumer thread: exception') # to show this line the sleep should be here not on main thread

logging.info("Main    : before creating threads")
x = threading.Thread(target=producer_function)
y = threading.Thread(target=consumer_function)
logging.info("Main    : before running thread")
x.start()
y.start()

11:50:03: Main    : before creating threads
11:50:03: Main    : before running thread
11:50:03: producer thread: starting
11:50:03: consumer thread: starting


In [6]:
# check if mongo received data
import sys
docs = list(myclient.nba.sunsbucks0.find())
len(docs)
#docs[-1]

11:50:03: Updated partition assignment: [TopicPartition(topic='sunsbucks0', partition=0)]
11:50:03: <BrokerConnection node_id=1001 host=e1c8653d4ed9:9092 <connecting> [IPv4 ('172.18.0.5', 9092)]>: connecting to e1c8653d4ed9:9092 [('172.18.0.5', 9092) IPv4]
11:50:03: <BrokerConnection node_id=1001 host=e1c8653d4ed9:9092 <connecting> [IPv4 ('172.18.0.5', 9092)]>: Connection complete.
11:50:03: <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connected> [IPv4 ('172.18.0.5', 9092)]>: Closing connection. 


1573

In [7]:
import pandas
docs = pandas.DataFrame(docs)
docs.tail()

11:50:05: <BrokerConnection node_id=1001 host=e1c8653d4ed9:9092 <connecting> [IPv4 ('172.18.0.5', 9092)]>: connecting to e1c8653d4ed9:9092 [('172.18.0.5', 9092) IPv4]
11:50:05: <BrokerConnection node_id=1001 host=e1c8653d4ed9:9092 <connecting> [IPv4 ('172.18.0.5', 9092)]>: Connection complete.
11:50:05: <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connected> [IPv4 ('172.18.0.5', 9092)]>: Closing connection. 


,_id,id,user_location,language,timestamp,text,retweet_count,user_followers,geo,coordinates,...,is_a_retweet,user_mentions,entities_hashtags,in_reply_to_status_id_str,in_reply_to_user_id_str,retweeted_status_id,id_str,screen_name,retweeted_status__user_id,retweeted_status_user_name
1568,60f6b82db614d8ed63e194a7,1417451455087202306,None,en,2021-07-20 13:48:56,#NBAFinals Match Prediction on #BSports!\n\n⚔️...,0,24468,None,None,...,"<Thread(Thread-5, started 140719684310784)>",[],"[{'text': 'NBAFinals', 'indices': [0, 10]}, {'...",None,None,NaN,1353605510524637185,BSports_io,NaN,NaN
1569,60f6b836b614d8ed63e194a8,1417451493842690050,"Wisconsin, USA",en,2021-07-20 13:49:05,RT @drdavemke: Good morning @Bucks fans. I thi...,0,63,None,None,...,"<Thread(Thread-5, started 140719684310784)>","[{'screen_name': 'drdavemke', 'name': 'David M...",[],None,None,1417427625346322434,630540817,btmotherbrain,3282825776,David Margolis
1570,60f6b839b614d8ed63e194a9,1417451503485394954,"Milwaukee, WI",en,2021-07-20 13:49:08,RT @ESPNMilwaukee: Good morning #Bucks fans…it...,0,70,None,None,...,"<Thread(Thread-5, started 140719684310784)>","[{'screen_name': 'ESPNMilwaukee', 'name': 'ESP...","[{'text': 'Bucks', 'indices': [32, 38]}, {'tex...",None,None,1417442493864480768,1382738758102044675,loving_mke,30797328,ESPN Milwaukee
1571,60f6b83ab614d8ed63e194aa,1417451508489129985,248 Tampa Ave Venice FL 34285,en,2021-07-20 13:49:09,NBA final jokes… @KingJames #nba #NBAFinals #N...,0,2169,None,None,...,"<Thread(Thread-5, started 140719684310784)>","[{'screen_name': 'KingJames', 'name': 'LeBron ...","[{'text': 'nba', 'indices': [28, 32]}, {'text'...",None,None,NaN,1296576066706907136,ShellbackCBD,NaN,NaN
1572,60f6b84ab614d8ed63e194ab,1417451576906686480,"New Jersey, USA",en,2021-07-20 13:49:25,Hell of a run we’re on right now guys\n\n15-4-...,0,1647,None,None,...,"<Thread(Thread-5, started 140719684310784)>",[],"[{'text': 'MLB', 'indices': [87, 91]}, {'text'...",None,None,NaN,1006231903605751808,JFirestoneMPSC,NaN,NaN


In [8]:
# save tweets as csv (just in case the pipeline had issues)
docs.to_csv('suns-bucks-G6.csv')